# How does Langchain Work?

### Simple Example
First, we import the required dependencies. Here, we will be using OpenAI as an AI model, and also getting the OpenAI API key from the environment variable.

In [3]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

Now we load the OpenAI model using `ChatOpenAI`, passing the API key along to it. Here, we use GPT 3.5.

In [7]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=api_key)

To simply pass questions to the OpenAI, we simply use `chat_model.invoke` and pass the prompt as a string. We return the `result.content` to simply return the message provided by the AI.

In [9]:
result = chat_model.invoke("hello chatgpt!")
print(result.content)

Hello! How can I assist you today?


### Chains

(cr. [LangChain Documentation](https://python.langchain.com/docs/expression_language/get_started))

We import the necessary dependencies.

In [10]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

For this example, we create a simple output parser that parses text and splits them at every `,`.

In [19]:
class CommaSeparatedListOutputParser(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(", ")
    
parser = CommaSeparatedListOutputParser()
parser.parse("hi, bye, chocolate")

['hi', 'bye', 'chocolate']

Here, we can see the use of a prompt template. This is the instructions we provide to the AI, without having to manually type it in the prompt every time. We create a `ChatPromptTemplate` from the provided information, and LangChain will pass this template to the AI model every time instead of the user having to type it in every time.

Here, we ask the AI to generate 5 objects in a comma separated list.

In [13]:
template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

chat_prompt

ChatPromptTemplate(input_variables=['text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant who generates comma separated lists.\nA user will pass in a category, and you should generate 5 objects in that category in a comma separated list.\nONLY return a comma separated list, and nothing more.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])

We create the chain using `|` to separate each step. This uses LangChain's LangChain Expression Language (LCEL). This feeds the output from one component as an input into the next component.

In this case, the output from `chat_prompt` is passed to the `chat_model`, and the output from the AI is passed to the parser.

In [20]:
chain = chat_prompt | chat_model | CommaSeparatedListOutputParser()
result = chain.invoke({"text": "colors"})
print(result)

['red', 'blue', 'green', 'yellow', 'purple']
